# Business Case #5 - Retail - Demand Forecasting

## Authors:
#### Débora Santos (m20200748),Pedro Henrique Medeiros (m20200742), Rebeca Pinheiro (m20201096)

#### Group D - D4B Consulting

In [2]:
#IMPORT LIBRARIES
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import seaborn as sns
from datetime import datetime
from itertools import product
from math import ceil
%matplotlib inline 
from collections import Counter
import matplotlib.cm as cm


#Models
from sklearn.model_selection import train_test_split
#from xgboost import XGBRegressor
#from xgboost import plot_importance



import warnings
warnings.filterwarnings("ignore")

# Seeting seaborn style
sns.set()

In [4]:
#import dataset in csv
df = pd.read_csv('df_demand.csv')

In [5]:
df.head()

,ProductName_ID,Point-of-Sale_ID,Date,Quantity
0,ProductName_649,POS_1,2017-03-04,2.0
1,ProductName_649,POS_1,2016-05-02,4.0
2,ProductName_649,POS_1,2016-10-24,2.0
3,ProductName_649,POS_1,2017-10-13,2.0
4,ProductName_649,POS_1,2017-10-14,2.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90748395 entries, 0 to 90748394
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   ProductName_ID    object 
 1   Point-of-Sale_ID  object 
 2   Date              object 
 3   Quantity          float64
dtypes: float64(1), object(3)
memory usage: 2.7+ GB


In [7]:
df = downcast1(df)

18.7% compressed


#### Clean data

In [8]:
df1 = df.copy()

### Feature Engineering

In [9]:
df1['Date'] = df1['Date'].astype('datetime64[ns]')

In [10]:
df1['WeekofYear'] = df1.Date.dt.weekofyear.astype(str)

In [11]:
df1['Year'] = df1.Date.dt.year.astype(str)

In [12]:
df1['Year_Week']= df1['Year'] + df1['WeekofYear'] 

In [13]:
df1['Year_Week']= df1['Year_Week'].astype(int)

In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90748395 entries, 0 to 90748394
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   ProductName_ID    object        
 1   Point-of-Sale_ID  object        
 2   Date              datetime64[ns]
 3   Quantity          int16         
 4   WeekofYear        object        
 5   Year              object        
 6   Year_Week         int32         
dtypes: datetime64[ns](1), int16(1), int32(1), object(4)
memory usage: 3.9+ GB


In [15]:
df1.drop(['Date'], axis=1, inplace=True)

In [16]:
df1.head()

,ProductName_ID,Point-of-Sale_ID,Quantity,WeekofYear,Year,Year_Week
0,ProductName_649,POS_1,2,9,2017,20179
1,ProductName_649,POS_1,4,18,2016,201618
2,ProductName_649,POS_1,2,43,2016,201643
3,ProductName_649,POS_1,2,41,2017,201741
4,ProductName_649,POS_1,2,41,2017,201741


In [17]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90748395 entries, 0 to 90748394
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   ProductName_ID    object
 1   Point-of-Sale_ID  object
 2   Quantity          int16 
 3   WeekofYear        object
 4   Year              object
 5   Year_Week         int32 
dtypes: int16(1), int32(1), object(4)
memory usage: 3.2+ GB


In [18]:
df_group = pd.DataFrame(df1.groupby(['Year_Week','Year', 'WeekofYear','ProductName_ID','Point-of-Sale_ID'])['Quantity'].sum())

In [19]:
df_group.reset_index(inplace = True)

In [20]:
df_group[['Position','Store_ID']] =(df_group['Point-of-Sale_ID']).str.split("_",expand = True)

In [21]:
df_group.drop(['Point-of-Sale_ID','Position'], axis=1, inplace=True)
df_group.head()

,Year_Week,Year,WeekofYear,ProductName_ID,Quantity,Store_ID
0,20161,2016,1,ProductName_1000,5,100
1,20161,2016,1,ProductName_1000,2,102
2,20161,2016,1,ProductName_1000,9,103
3,20161,2016,1,ProductName_1000,7,104
4,20161,2016,1,ProductName_1000,3,106


In [22]:
df_group[['Product','Product_ID']] =(df_group['ProductName_ID']).str.split("_",expand = True)

In [23]:
df_group.drop(['ProductName_ID','Product'], axis=1, inplace=True)
df_group.head()

,Year_Week,Year,WeekofYear,Quantity,Store_ID,Product_ID
0,20161,2016,1,5,100,1000
1,20161,2016,1,2,102,1000
2,20161,2016,1,9,103,1000
3,20161,2016,1,7,104,1000
4,20161,2016,1,3,106,1000


In [24]:
df_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29318460 entries, 0 to 29318459
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Year_Week   int64 
 1   Year        object
 2   WeekofYear  object
 3   Quantity    int16 
 4   Store_ID    object
 5   Product_ID  object
dtypes: int16(1), int64(1), object(4)
memory usage: 1.1+ GB


In [25]:
df_group = downcast1(df_group)

9.5% compressed


In [28]:
df_group.to_csv('df_group_exported.csv')

In [26]:
lista = list(set(df_group['Year_Week']))

In [27]:
# Create a dataframe of the Cartesian Product of the unique stores and unique products for each week

from itertools import product

train = []

for i in lista:
    
    stores = df_group.loc[df_group["Year_Week"] == i, "Store_ID"].unique()
    
    products = df_group.loc[df_group["Year_Week"] == i, "Product_ID"].unique()
    
    train.append(np.array(list(product(*[[i],stores, products]))))
    
index_feats = ["Year_Week", "Store_ID", "Product_ID"]

train = pd.DataFrame(np.vstack(train), columns=index_feats)

MemoryError: Unable to allocate 14.8 GiB for an array with shape (120361747, 3) and data type <U11

In [ ]:
# Create the column showing how many of each product have been sold in each week

group = df_group.groupby(index_feats).agg({"Quantity": "sum"})
group = group.reset_index()

train = pd.merge(train, group, on=index_feats, how="left")
train

In [ ]:
# Use garbage collection to minimise memory usage

import gc

del group

gc.collect()

In [ ]:
train = train.fillna(0)

In [ ]:
train.head()

In [ ]:
test = train[train['Year_Week']>=201938]

In [ ]:
train = train[train['Year_Week']<201938]

In [ ]:
test.head()

In [ ]:
test2 = test.drop('Quantity',axis = 1)
test2

In [ ]:
df_concat = pd.concat([train, test2], ignore_index=True, keys=index_feats)

In [ ]:
df_concat.head()

In [ ]:
df_concat[df_concat['Year_Week']==201938]

In [ ]:
df_forecast = df_concat.fillna(0)
df_forecast.head()

In [ ]:
df_forecast = downcast1(df_forecast)

In [ ]:
def add_mean_feats(df, mean_feats, index_features, agg_col="Quantity", agg_func="mean"):
    
    """
    Function to automatically create new features showing the mean of quantity grouped by the specified columns.
    """
    
    if len(index_features) == 2:
        feature_name = index_features[1] + f"_{agg_col}_{agg_func}"
    else: 
        feature_name = index_features[1] + "_" + index_features[2] + f"_{agg_col}_{agg_func}"
        
    group = df.groupby(index_features).agg({agg_col:agg_func}).reset_index().rename(columns={agg_col:feature_name})
    
    df = pd.merge(df, group, on=index_features, how="left")
    
    df = downcast1(df)
    
    mean_feats.append(feature_name)
    
    del group
    gc.collect()
    
    return df, mean_feats

In [ ]:
prod_mean_features = []

df_forecast, prod_mean_features = add_mean_feats(df_forecast, prod_mean_features, ["Year_Week","Product_ID"])

df_forecast

In [ ]:
df_forecast, prod_mean_features = add_mean_feats(df_forecast, prod_mean_features, ["Year_Week","Product_ID", 'Store_ID'])

df_forecast

In [ ]:
store_mean_features = []
df_forecast, store_mean_features = add_mean_feats(df_forecast, store_mean_features, ["Year_Week", 'Store_ID'])

df_forecast

In [ ]:
def add_lags(df, lag_features, index_features, lag_feature, lags=[1,2,3], clip=False):
    
    """
    Function to automatically create lag features based on the columns specified.
    """
    
    df_temp = df[index_features + [lag_feature]].copy()
    
    for i in lags:
        
        feat_name = lag_feature + "_lag" + str(i)
        df_temp.columns = index_features + [feat_name]
        df_temp["Year_Week"] += i
        df = pd.merge(df, df_temp.drop_duplicates(), on=index_features, how="left")
        df[feat_name] = df[feat_name].fillna(0)
        
        if clip:
            lag_feats_to_clip.append(feat_name)
            
    df = downcast1(df)
    del df_temp
    gc.collect()
    
    return df, lag_feats_to_clip

In [ ]:
lag_feats_to_clip = []
index_features = ["Year_Week", 'Store_ID', "Product_ID"]

df_forecast, lag_feats_to_clip = add_lags(df_forecast, lag_feats_to_clip, index_features, "Quantity", clip=True)

In [ ]:
df_forecast.head()

In [ ]:
# Now use the lists that have saved previously in creating the mean features to create additional lags

for item in prod_mean_features:
    
    df_forecast, lag_feats_to_clip = add_lags(df_forecast, lag_feats_to_clip, index_features, item, clip=True)

In [ ]:
for item in store_mean_features:
    
    df_forecast, lag_feats_to_clip = add_lags(df_forecast, lag_feats_to_clip, index_features, item, clip=True)

In [ ]:
df_forecast

In [ ]:
# Create feature showing mean of the three lags

df_forecast["Quantity_lag3_mean"] = df_forecast[["Quantity_lag1", "Quantity_lag2", "Quantity_lag3"]].mean(axis=1)

In [ ]:
df_forecast[lag_feats_to_clip + ["Quantity_lag3_mean", "Quantity"]] =  df_forecast[lag_feats_to_clip + ["Quantity_lag3_mean", "Quantity"]].clip(0,20)

In [ ]:
df_forecast["lag_grad_1"] = df_forecast["Quantity_lag1"] / df_forecast["Quantity_lag2"]
df_forecast["lag_grad_1"] = df_forecast["lag_grad_1"].replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:
df_forecast["lag_grad_2"] = df_forecast["Quantity_lag2"] / df_forecast["Quantity_lag3"]
df_forecast["lag_grad_2"] = df_forecast["lag_grad_2"].replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:
df_forecast = downcast1(df_forecast)
df_forecast.info()

In [ ]:
df_forecast.to_csv(os.path.join("df_demand_complete.csv"), index=False)

In [ ]:
test.to_csv(os.path.join("teste.csv"), index=False)